## Instalação:

Para a instalar a biblioteca do Selenium e Webdriver Manager abra o prompt de comando e execute o comando:


```
pip install --upgrade selenium
```

```
pip install webdriver_manager
```

Documentação selenium: https://selenium-python.readthedocs.io/index.html

Sempre verifique a versão do Chrome instalado na sua máquina em: ajuda > Sobre o Google Chrome. <br>
Baixar a versão do webdriver correspondente a que você esteja utilizando no Chrome:<br>
https://chromedriver.chromium.org/downloads

In [3]:
# Importando as bibliotecas 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By # classe BY
import pandas as pd

# Abrindo navegador e selecionando chromedriver.exe na pasta local do computador
navegador = webdriver.Chrome('C:\Program Files\Google\Chrome\chromedriver.exe')

# Abrindo o site desejado - Mega Sena
link = ('https://loterias.caixa.gov.br/Paginas/Mega-Sena.aspx')
navegador.get(link)
navegador.maximize_window()

C:\Users\fdgce\AppData\Local\Temp\ipykernel_18496\36597335.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navegador = webdriver.Chrome('C:\Program Files\Google\Chrome\chromedriver.exe')


A classe By serve para que possamos indentificar o elemento dentro do HTML. 
Existem várias formas de encontrar o elemento:
```
ID = "id"
NAME = "name"
XPATH = "xpath"
LINK_TEXT = "link text"
PARTIAL_LINK_TEXT = "partial link text"
TAG_NAME = "tag name"
CLASS_NAME = "class name"
CSS_SELECTOR = "css selector"
```

Exemplos:
```
find_element(By.ID, "id")
find_element(By.NAME, "name")
find_element(By.XPATH, "xpath")
find_element(By.LINK_TEXT, "link text")
find_element(By.PARTIAL_LINK_TEXT, "partial link text")
find_element(By.TAG_NAME, "tag name")
find_element(By.CLASS_NAME, "class name")
find_element(By.CSS_SELECTOR, "css selector")
```

In [4]:
# Pegando os campos do HTML pelo Xpath

# Informações sobre concurso e data
info_concurso = navegador.find_element(By.XPATH, '//*[@id="wp_resultados"]/div[1]/div/h2/span').text

# Criando a lista vazia para pegar as dezenas
numeros = ['', '', '', '', '', ''] 
j = 0

for i in range(6):
    i = i+1
    numeros[j] = navegador.find_element(By.XPATH, f'//*[@id="ulDezenas"]/li[{i}]').text
    j = j+1

Pegamos a string com concurso e data, transformamos a informação, dividimos e colocamos em variáveis:

In [5]:
concurso = info_concurso[9:13]
data = info_concurso[-11:-1]

In [6]:
print(info_concurso)
print("Dezenas:", numeros)
print("Concurso:", concurso)
print("Data:", data)


Concurso 2544 (30/11/2022)
Dezenas: ['25', '38', '45', '53', '55', '56']
Concurso: 2544
Data: 30/11/2022


Com as informações sobre concurso, data e dezenas, vamos criar nosso Data Frame:

In [7]:
sorteios = pd.DataFrame(columns = ["Data", "Concurso", "Num 1", "Num 2", "Num 3", "Num 4", "Num 5", "Num 6"])
sorteios

,Data,Concurso,Num 1,Num 2,Num 3,Num 4,Num 5,Num 6


Inserindo as informações no Data Frame


In [8]:
sorteios.loc[len(sorteios)] = [data, concurso, numeros[0], numeros[1], numeros[2], numeros[3], numeros[4], numeros[5]]
sorteios

,Data,Concurso,Num 1,Num 2,Num 3,Num 4,Num 5,Num 6
0,30/11/2022,2544,25,38,45,53,55,56


Vamos navegar pelo HTML e selecionar o resultado de outro concurso:

In [9]:
# Selecionando um sorteio aleatório
numero_sorteio = 1000

# Comando para clicar na caixa de busca dos sorteios
navegador.find_element(By.XPATH, '//*[@id="buscaConcurso"]').click()

# Comando para escrever na caixa de busca e pressionar enter
navegador.find_element(By.XPATH, '//*[@id="buscaConcurso"]').send_keys(numero_sorteio, Keys.ENTER)

In [10]:
# Informações sobre concurso e data
info_concurso = navegador.find_element(By.XPATH, '//*[@id="wp_resultados"]/div[1]/div/h2/span').text

# zerando nosso contador da lista de números
j = 0

# Inserindo as dezenas na lista
for i in range(6):
    i = i+1
    numeros[j] = navegador.find_element(By.XPATH, f'//*[@id="ulDezenas"]/li[{i}]').text
    j = j+1


In [11]:
# Pegando as informações e colocando em variáveis
concurso = info_concurso[9:13]
data = info_concurso[-11:-1]

In [12]:
# Exibindo as informações
print(info_concurso)
print("Dezenas:", numeros)
print("Concurso:", concurso)
print("Data:", data)


Concurso 1000 (30/08/2008)
Dezenas: ['29', '38', '39', '49', '53', '58']
Concurso: 1000
Data: 30/08/2008


In [13]:
# Inserindo as informações do concurso 1000 no Data Frame
sorteios.loc[len(sorteios)] = [data, concurso, numeros[0], numeros[1], numeros[2], numeros[3], numeros[4], numeros[5]]
sorteios

,Data,Concurso,Num 1,Num 2,Num 3,Num 4,Num 5,Num 6
0,30/11/2022,2544,25,38,45,53,55,56
1,30/08/2008,1000,29,38,39,49,53,58


## RESUMO:

A primeira parte do código foi um explicativo de como funciona a raspagem com o Selenium. O código abaixo é o que vamos utilizar daqui pra frente.

In [1]:
# Importando as bibliotecas 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By # classe BY
import pandas as pd
import time

# Abrindo navegador e selecionando chromedriver.exe na pasta local do computador
navegador = webdriver.Chrome('C:\Program Files\Google\Chrome\chromedriver.exe')

# Abrindo o site desejado - Mega Sena
link = ('https://loterias.caixa.gov.br/Paginas/Mega-Sena.aspx')
navegador.get(link)
navegador.maximize_window()

C:\Users\fdgce\AppData\Local\Temp\ipykernel_20348\881746509.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navegador = webdriver.Chrome('C:\Program Files\Google\Chrome\chromedriver.exe')


In [2]:
sorteios = pd.DataFrame(columns = ["Data", "Concurso", "Num 1", "Num 2", "Num 3", "Num 4", "Num 5", "Num 6"])
sorteios

,Data,Concurso,Num 1,Num 2,Num 3,Num 4,Num 5,Num 6


Fizemos um looping, iniciando do 1 até o último sorteio. Tratamos o tamanho da String no caso dos sorteios de 1 até 9, 10 até 99 e 100 até 999:

In [3]:
info_concurso = navegador.find_element(By.XPATH, '//*[@id="wp_resultados"]/div[1]/div/h2/span').text
concurso = int(info_concurso[9:13])
inicio_contador = concurso

# Pegando as dezenas
numeros = ['', '', '', '', '', ''] 

for cont in range(inicio_contador + 1):
    j = 0
    cont = cont + 1
    navegador.find_element(By.XPATH, '//*[@id="buscaConcurso"]').click()
    navegador.find_element(By.XPATH, '//*[@id="buscaConcurso"]').clear()
    navegador.find_element(By.XPATH, '//*[@id="buscaConcurso"]').send_keys(cont)
    time.sleep(0.5)
    navegador.find_element(By.XPATH, '//*[@id="buscaConcurso"]').send_keys(Keys.ENTER)
    data = info_concurso[-11:-1]

    # Até o 9º sorteio len = 23
    if len(info_concurso) == 23:
        concurso = int(info_concurso[9])

    # Até o 99º sorteio len = 24
    elif len(info_concurso) == 24:
        concurso = int(info_concurso[9:11])
    
    # Até o 999º sorteio len = 25
    elif len(info_concurso) == 25:
        concurso = int(info_concurso[9:12])
    
    # Acima de 999 len = 26
    elif len(info_concurso) == 26:
         concurso = int(info_concurso[9:13])

    for i in range(6):
        i = i+1
        time.sleep(0.5)
        numeros[j] = int(navegador.find_element(By.XPATH, f'//*[@id="ulDezenas"]/li[{i}]').text)
        j += 1
    
    sorteios.loc[len(sorteios)] = [data, concurso, numeros[0], numeros[1], numeros[2], numeros[3], numeros[4], numeros[5]]
    info_concurso = navegador.find_element(By.XPATH, '//*[@id="wp_resultados"]/div[1]/div/h2/span').text
    



Depois de rodarmos nosso código, vamos ver as informações do Data Frame

In [4]:
sorteios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2544 entries, 0 to 2543
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Data      2544 non-null   object
 1   Concurso  2544 non-null   int64 
 2   Num 1     2544 non-null   int64 
 3   Num 2     2544 non-null   int64 
 4   Num 3     2544 non-null   int64 
 5   Num 4     2544 non-null   int64 
 6   Num 5     2544 non-null   int64 
 7   Num 6     2544 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 178.9+ KB


In [5]:
sorteios

,Data,Concurso,Num 1,Num 2,Num 3,Num 4,Num 5,Num 6
0,30/11/2022,2544,4,5,30,33,41,52
1,11/03/1996,1,9,37,39,41,43,49
2,18/03/1996,2,10,11,29,30,36,47
3,25/03/1996,3,1,5,6,27,42,59
4,01/04/1996,4,1,2,6,16,19,46
...,...,...,...,...,...,...,...,...
2539,16/11/2022,2539,2,8,28,34,41,49
2540,19/11/2022,2540,10,28,45,47,57,59
2541,22/11/2022,2541,12,20,22,25,26,55
2542,24/11/2022,2542,2,5,27,30,46,53


Depois de quase duas horas, conseguimos pegar todos os dados, não esqueça de salvar

In [6]:
sorteios.to_csv(r'C:\Users\fdgce\OneDrive\Área de Trabalho\Estudos\Python\Manual web scraping\webScrapingMegaSena\resultado.csv', index = False)

## Tratamento dos dados

Como pode ver, a primeira linha contém o resultado do último sorteio, ficou fora de ordem. Iremos tratar nosso Data Frame a seguir

Em construção...